<a href="https://colab.research.google.com/github/songseokbeom/Kaggle/blob/master/Plant%20Pathology%202020%20-%20FGVC7/KNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle Plant Pathology 2020 - FGVC7(Detecting the category of foliar diseases in apple trees)

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [0]:
MAIN_PATH = "../input/plant-pathology-2020-fgvc7"
print(os.listdir(MAIN_PATH))

IMAGE_PATH = MAIN_PATH + "/images/"
train_df = pd.read_csv(MAIN_PATH + "/train.csv")
test_df = pd.read_csv(MAIN_PATH + "/test.csv")

In [0]:
# 이미지 갯수, csv shape 확인
print('train data shape: ', train_df.shape)
print('Total images in train set: ', train_df['image_id'].count())
print('[train_csv example]\n', train_df.head(3))
print('---------------------------------------------------')
print('test data shape: ', test_df.shape)
print('Total images in test set: ', train_df['image_id'].count())
print('[test_csv example]\n', test_df.head(3))

In [0]:
# 분류되지 않은 데이터 확인
print('train set')
print(train_df.info())
print('------------------------------------------')
print('test set')
print(test_df.info())

In [0]:
# class 추출
temp=[]
classes = {}
for col in train_df.columns:
    temp.append(col) 
temp.remove('image_id')
for i in range(len(temp)):
    classes[i] = temp[i]

In [0]:
# 클래스 별 샘플 수 체크
for c in range(0,len(classes)):
    print(f"#{classes[c]} samples: {train_df[classes[c]].sum()}")

multiple_diseases 샘플 수가 다른 샘플들에 비해 상대적으로 적음

In [0]:
# 중복된 데이터 확인
train_id = set(train_df.image_id.values)
print(f"#Unique train images: {len(train_id)}")
test_id = set(test_df.image_id.values)
print(f"#Unique train images: {len(test_id)}")
both_images = train_id.intersection(test_id)
print(f"#Images in both train set and test set: {len(both_images)}")

train set, test set내에 중복된 데이터는 존재하지 않음
또한, train set과 test set에 모두 들어간 데이터도 존재하지 않음

In [0]:
train_X = train_df.sample(n=100)
train_Y = np.zeros((100))
test_X = test_df.sample(n=50)
train_X_flatten, test_X_flatten = [], []

for i in range(0, len(train_X)):
    s = train_X[i:i+1].values
    img = np.array(Image.open(IMAGE_PATH + s[0][0] +'.jpg'))
    train_X_flatten.append(np.ravel(img, order='C'))
    for j in range(1, 5):
        if(s[0][j]==1):
            train_Y[i] = j - 1
            break
train_X_flatten = np.array(train_X_flatten)

for i in range(0, len(test_X)):
    s = test_X[i:i+1].values
    img = np.array(Image.open(IMAGE_PATH + s[0][0] + '.jpg'))
    test_X_flatten.append(np.ravel(img, order='C'))
test_X_flatten = np.array(test_X_flatten)

print(f"Train set(flatten) shape: {train_X_flatten.shape}")
print(f"Test set(flatten) shape: {test_X_flatten.shape}")

In [0]:
scaler = StandardScaler()
scaler.fit(train_X_flatten)
train_X_std = scaler.transform(train_X_flatten)
test_X_std = scaler.transform(test_X_flatten)

In [0]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(train_X_std, train_Y)

In [0]:
score = neigh.score(train_X_std, train_Y)
print(f"Model accuracy: {score}")

In [0]:
os.chdir(r'kaggle/working')
from IPython.display import FileLink
FileLink(r'df_name.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'kaggle/working'